In [1]:
import os
import cv2
import json
import shutil
import random
import pickle
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
from pprint import pprint
import matplotlib.pyplot as plt

In [2]:
def normalize_image(image, hdr_mode=False, normalization_policy='percentile'):
    if image.dtype == np.uint8:
        normalization_policy = "default"
    if normalization_policy.startswith("percentile") and hdr_mode:
        if image.dtype != np.float32 and image.dtype != np.uint32:
            raise ValueError('HDR image type is {} instead of float32 or uint32'.format(image.dtype))
        percentile_normalization_lower_bound = 0.1
        percentile_normalization_upper_bound = 99.5

        if normalization_policy == "percentile":
            lower_bound = np.array([np.percentile(image[..., i],
                                                  percentile_normalization_lower_bound,
                                                  interpolation='lower')
                                    for i in range(3)])
            upper_bound = np.array([np.percentile(image[..., i],
                                                  percentile_normalization_upper_bound,
                                                  interpolation='lower')
                                    for i in range(3)])
        elif normalization_policy == "percentile_vpu":
            r, g, b = image[..., 0], image[..., 1], image[..., 2]
            brightness = (3 * r + b + 4 * g) / 8
            lower_bound = np.percentile(brightness, percentile_normalization_lower_bound,
                                        interpolation='lower')
            upper_bound = np.percentile(brightness, percentile_normalization_upper_bound,
                                        interpolation='lower')

        image = (image.astype(np.float32) - lower_bound) / (upper_bound - lower_bound)
        image = np.clip(image, 0.0, 1.0)
    elif normalization_policy == "3sigma" and hdr_mode:
        if image.dtype != np.float32 and image.dtype != np.uint32:
            raise ValueError('HDR image type is {} instead of float32 or uint32'.format(image.dtype))
        sigma_size = 3
        min_variance = 1200
        r, g, b = image[...,0], image[...,1], image[...,2]
        brightness = (3 * r + b + 4 * g) / 8
        mean, sigma = np.mean(brightness), np.std(brightness)
        brightness_min, brightness_max = np.min(brightness), np.max(brightness)
        if (sigma * sigma_size) > mean:
            lmin = brightness_min
            lmax = min(brightness_max, mean * sigma_size)
            if (lmax - lmin) < min_variance:
                lmax = lmin + min_variance
            lower_bound = lmin
            upper_bound = lmax
        else:
            mean_var = mean - sigma_size * sigma
            output_min = max(brightness_min, mean_var)
            mean_var = mean + sigma_size * sigma
            output_max = min(brightness_max, mean_var)
            if (output_max - output_min) < min_variance:
                output_min = mean - min_variance / 2.0
                output_min = 0 if output_min < 0 else output_min
                output_max = output_min + min_variance
            lower_bound = output_min
            upper_bound = output_max
        image = (image.astype(np.float32) - lower_bound) / (upper_bound - lower_bound)
        image = np.clip(image, 0.0, 1.0)
    elif normalization_policy == 'tonemap' and hdr_mode:
        if image.dtype != np.float32 and image.dtype != np.uint32:
            raise ValueError('HDR image type is {} instead of float32 or uint32'.format(image.dtype))
        alpha = 0.5
        r, g, b = image[...,0], image[...,1], image[...,2]
        L_in = 0.27 * r + 0.67 * g + 0.06 * b
        Lw = np.exp(np.mean(np.log(L_in + 1e-6)))
        n = alpha * L_in / Lw
        L_out = n / (1 + n) * (1 + n / (n.max() ** 2))
        image = np.clip(image.astype(np.float32) * (L_out / L_in)[..., None], 0.0, 1.0)
    elif normalization_policy == "none" and hdr_mode:
        lower_bound = 0.0
        upper_bound = 2**20 - 1
        image = (image.astype(np.float32) - lower_bound) / (upper_bound - lower_bound)
        image = np.clip(image, 0.0, 1.0)
    elif normalization_policy == "default" or not hdr_mode:
        assert np.max(image) <= 255 and np.min(image) >= 0, \
            "Image with linear model should be in range [0,255]"
        lower_bound = 0.0
        upper_bound = 255.0
        image = (image.astype(np.float32) - lower_bound) / (upper_bound - lower_bound)
        image = np.clip(image, 0.0, 1.0)
    else:
        raise ValueError("normalization_policy is not supported!")
    return image

def plot_stereo_images(df, key1, key2):
    print('Tone mapped left image |', 'Percentile normalized left image |', 'Percentile VPU normalized left image')
    print('Tone mapped right image |', 'Percentile normalized right image |', 'Percentile VPU normalized right image')
    for i, sample_df in df.iterrows():
        data_path = os.path.join(root_dir, 'processed/images', sample_df.image_id, 'stereo_output.npz')
        data = np.load(data_path)
        img_left = data['left']
        img_tonemap_left = normalize_image(img_left, sample_df.hdr_mode, 'tonemap')
        img_pcntvpu_left = normalize_image(img_left, sample_df.hdr_mode, 'percentile_vpu')
        img_left = normalize_image(img_left, sample_df.hdr_mode)
        img_right = data['right']
        img_tonemap_right = normalize_image(img_right, sample_df.hdr_mode, 'tonemap')
        img_pcntvpu_right = normalize_image(img_right, sample_df.hdr_mode, 'percentile_vpu')
        img_right = normalize_image(img_right, sample_df.hdr_mode)
        
        print(sample_df.image_id, key1, sample_df[key1], key2, sample_df[key2])
        plt.figure(1, figsize=(34, 12))
        plt.subplot(231)
        plt.imshow(img_tonemap_left)
        plt.subplot(232)
        plt.imshow(img_left)
        plt.subplot(233)
        plt.imshow(img_pcntvpu_left)
        plt.subplot(234)
        plt.imshow(img_tonemap_right)
        plt.subplot(235)
        plt.imshow(img_right)
        plt.subplot(236)
        plt.imshow(img_pcntvpu_right)
        plt.show()

In [3]:
root_dir = '/data/jupiter/datasets/Jupiter_train_v4_71'
data_dir = os.path.join(root_dir, 'images')
df = pd.read_csv(os.path.join(root_dir, 'master_annotations.csv'), low_memory=False)
# df = df[df.hdr_mode == True]
df.shape

(195289, 234)

In [4]:
# iq_csv = '/data/jupiter/li.yu/data/Jupiter_train_v4_53/v453_iq.csv'
iq_csv = '/data/jupiter/li.yu/data/Jupiter_train_v4_71/v471_iq.csv'
iq_df = pd.read_csv(iq_csv)
print(iq_df.shape, iq_df[iq_df.iq != 'good'].shape)
bad_iq_df = iq_df[iq_df.iq != 'good']
iq_df.groupby('iq').count()

(195289, 11) (17846, 11)


In [5]:
iq_df.head(2)

,image_id,iq,iq_features,iq_features_total,iq_features_low,iq_features_mid,iq_features_high,iq_features_sharpness,iq_features_smudge,iq_features_smudge_reason,iq_process_time
0,614d0eb37585e5bf211d83dc,smudge,"{'confidence': 1.0, 'image_features': {'total'...",1572864,0.0,1572864.0,0.0,2140.6290,108.098389,pixel_difference_on_gray_image,0.124736
1,614d0eb90f7787d7ba7d1e43,good,"{'confidence': 1.0, 'image_features': {'total'...",1572864,0.0,1572864.0,0.0,4105.3945,33.200928,not_smudged,0.067164


In [6]:
iq_df.iloc[0].iq_features

"{'confidence': 1.0, 'image_features': {'total': 1572864, 'low': 0.0, 'mid': 1572864, 'high': 0.0, 'sharpness': 2140.629, 'smudge': 108.098388671875, 'smudge_reason': 'pixel_difference_on_gray_image'}}"

In [7]:
iq_df.iloc[1].iq_features

"{'confidence': 1.0, 'image_features': {'total': 1572864, 'low': 0.0, 'mid': 1572864, 'high': 0.0, 'sharpness': 4105.3945, 'smudge': 33.200927734375, 'smudge_reason': 'not_smudged'}}"

In [8]:
print(iq_df[iq_df.iq == 'dark'].shape, iq_df[iq_df.iq_features_low > 0.6].shape)
print(iq_df[iq_df.iq == 'bright'].shape, iq_df[iq_df.iq_features_high > 0.18].shape)

(11715, 11) (11948, 11)
(3297, 11) (3489, 11)


In [9]:
iq_df.groupby('iq_features_smudge_reason').count()

,image_id,iq,iq_features,iq_features_total,iq_features_low,iq_features_mid,iq_features_high,iq_features_sharpness,iq_features_smudge,iq_process_time
iq_features_smudge_reason,,,,,,,,,,
adaptive_threshold,307,307,307,307,307,307,307,307,307,307
blur,304,304,304,304,304,304,304,304,304,304
not_smudged,192455,192455,192455,192455,192455,192455,192455,192455,192455,192455
pixel_difference_on_binary_image,410,410,410,410,410,410,410,410,410,410
pixel_difference_on_gray_image,1813,1813,1813,1813,1813,1813,1813,1813,1813,1813


In [10]:
print(iq_df[iq_df.iq == 'smudge'].shape, iq_df[(iq_df.iq_features_smudge_reason == 'pixel_difference_on_gray_image') | (iq_df.iq_features_smudge_reason == 'pixel_difference_on_binary_image') | (iq_df.iq_features_smudge_reason == 'adaptive_threshold') | (iq_df.iq_features_smudge_reason == 'blur')].shape)
print(iq_df[iq_df.iq_features_smudge_reason == 'pixel_difference_on_gray_image'].shape, iq_df[(iq_df.iq_features_smudge > 88) & (iq_df.iq_features_smudge_reason == 'pixel_difference_on_gray_image')].shape)
print(iq_df[iq_df.iq_features_smudge_reason == 'pixel_difference_on_binary_image'].shape, iq_df[(iq_df.iq_features_smudge > 40) & (iq_df.iq_features_smudge_reason == 'pixel_difference_on_binary_image')].shape)
print(iq_df[iq_df.iq_features_smudge_reason == 'adaptive_threshold'].shape, iq_df[(iq_df.iq_features_smudge > 8500) & (iq_df.iq_features_smudge < 458429) & (iq_df.iq_features_smudge_reason == 'adaptive_threshold')].shape)
print(iq_df[iq_df.iq_features_smudge_reason == 'blur'].shape, iq_df[(iq_df.iq_features_smudge < 15) & (iq_df.iq_features_smudge_reason == 'blur')].shape)


(2834, 11) (2834, 11)
(1813, 11) (1813, 11)
(410, 11) (410, 11)
(307, 11) (307, 11)
(304, 11) (251, 11)


## Bright images with iq_features_high value in range [0.18, 0.2]

In [5]:
iq_df = iq_df.merge(df, on='image_id')

,image_id,iq_features,iq_features_total,iq_features_low,iq_features_mid,iq_features_high,iq_features_sharpness,iq_features_smudge,iq_features_smudge_reason,iq_process_time
iq,,,,,,,,,,
bright,3297,3297,3297,3297,3297,3297,3297,3297,3297,3297
dark,11715,11715,11715,11715,11715,11715,11715,11715,11715,11715
good,177443,177443,177443,177443,177443,177443,177443,177443,177443,177443
smudge,2834,2834,2834,2834,2834,2834,2834,2834,2834,2834


In [12]:
sub_df = iq_df[(iq_df.iq_features_high > 0.18) & (iq_df.iq_features_high < 0.2)]
print(sub_df.shape)
plot_stereo_images(sub_df.sample(5), 'iq', 'iq_features_high')

(667, 244)


## Bright images with iq_features_high value bigger than 0.2

In [13]:
sub_df = iq_df[(iq_df.iq_features_high > 0.2)]
print(sub_df.shape)
plot_stereo_images(sub_df.sample(5), 'iq', 'iq_features_high')

(2822, 244)


## Dark images with iq_features_low value in range [0.6, 0.8]

In [14]:
sub_df = iq_df[(iq_df.iq_features_low > 0.6) & (iq_df.iq_features_low < 0.8)]
print(sub_df.shape)
plot_stereo_images(sub_df.sample(5), 'iq', 'iq_features_low')

(3808, 244)


## Dark images with iq_features_low value bigger than 0.8

In [15]:
sub_df = iq_df[(iq_df.iq_features_low > 0.8)]
print(sub_df.shape)
plot_stereo_images(sub_df.sample(5), 'iq', 'iq_features_low')

(8140, 244)


## Smudge (on gray) value in range [88, 95]

In [20]:
sub_df = iq_df[(iq_df.iq_features_smudge > 88) & (iq_df.iq_features_smudge < 95) & (iq_df.iq_features_smudge_reason == 'pixel_difference_on_gray_image')]
print(sub_df.shape)
plot_stereo_images(sub_df.sample(5), 'iq_features_smudge_reason', 'iq_features_smudge')

(310, 244)


## Smudge (on gray) value bigger than 95

In [21]:
sub_df = iq_df[(iq_df.iq_features_smudge > 95) & (iq_df.iq_features_smudge_reason == 'pixel_difference_on_gray_image')]
print(sub_df.shape)
plot_stereo_images(sub_df.sample(5), 'iq_features_smudge_reason', 'iq_features_smudge')

(1503, 244)


## Smudge (on binary)

In [27]:
sub_df = iq_df[(iq_df.iq_features_smudge > 40) & (iq_df.iq_features_smudge_reason == 'pixel_difference_on_binary_image')]
print(sub_df.shape)
plot_stereo_images(sub_df.sample(5), 'iq_features_smudge_reason', 'iq_features_smudge')

(410, 244)


## Smudge (on adaptive_threshold)

In [28]:
sub_df = iq_df[(iq_df.iq_features_smudge > 8500) & (iq_df.iq_features_smudge < 458429) & (iq_df.iq_features_smudge_reason == 'adaptive_threshold')]
print(sub_df.shape)
plot_stereo_images(sub_df.sample(5), 'iq_features_smudge_reason', 'iq_features_smudge')

(307, 244)


## Smudge (on blur)

In [29]:
sub_df = iq_df[(iq_df.iq_features_smudge < 15) & (iq_df.iq_features_smudge_reason == 'blur')]
print(sub_df.shape)
plot_stereo_images(sub_df.sample(5), 'iq_features_smudge_reason', 'iq_features_smudge')

(251, 244)


## Combine sub_dfs with custom thresholds

In [10]:
sub_df1 = iq_df[(iq_df.iq_features_high > 0.2)]
sub_df2 = iq_df[(iq_df.iq_features_low > 0.9)]
sub_df3 = iq_df[(iq_df.iq_features_smudge > 90) & (iq_df.iq_features_smudge_reason == 'pixel_difference_on_gray_image')]
sub_df4 = iq_df[(iq_df.iq_features_smudge > 42) & (iq_df.iq_features_smudge_reason == 'pixel_difference_on_binary_image')]
sub_df5 = iq_df[(iq_df.iq_features_smudge > 8500) & (iq_df.iq_features_smudge < 458429) & (iq_df.iq_features_smudge_reason == 'adaptive_threshold')]
sub_df6 = iq_df[(iq_df.iq_features_smudge < 15) & (iq_df.iq_features_smudge_reason == 'blur')]
print(sub_df1.shape, sub_df2.shape, sub_df3.shape, sub_df4.shape, sub_df5.shape, sub_df6.shape)
bad_iq_df = pd.concat([sub_df1, sub_df2, sub_df3, sub_df4, sub_df5, sub_df6], ignore_index=True)
print(bad_iq_df.shape)
bad_iq_df.groupby(['operation_time']).count()

(2822, 244) (5426, 244) (1718, 244) (357, 244) (307, 244) (251, 244)
(10881, 244)


,image_id,iq,iq_features,iq_features_total,iq_features_low,iq_features_mid,iq_features_high,iq_features_sharpness,iq_features_smudge,iq_features_smudge_reason,...,label_map,label_save_path,median_depth_check,rectified_label_save_path,stereo_left_image,stereo_pipeline_npz_save_path,stereo_right_image,jupiter_interface_foxy_branch,jupiter_interface_foxy_hash,jupiter_metadata_schema
operation_time,,,,,,,,,,,,,,,,,,,,,
dawn_dusk,717,717,717,717,717,717,717,717,717,717,...,717,717,717,717,717,717,717,2,2,2
daytime,4390,4390,4390,4390,4390,4390,4390,4390,4390,4390,...,4390,4390,4390,4390,4390,4390,4390,0,0,0
nightime,5314,5314,5314,5314,5314,5314,5314,5314,5314,5314,...,5314,5314,5314,5314,5314,5314,5314,21,21,21
unknown,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,0,0,0
